# Extract PDF
This Script tries to detect, when someone was speeching and extracts them as raw as possible.

In [79]:
import pandas as pd
import json
from pathlib import Path
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar

In [ ]:
def read_pdf(f):
    i = 0

    textbuf = ""
    reading = False

    paragraphs = []

    fakes = ['Detailberatung', 'Titel und Ingress', 'I. und II.', 'III.', 'Abstimmung', 'IV.',
    'Schlussabstimmung']

    pages = extract_pages(f)

    i = 0
    for page_layout in pages:
        i += 1
        #if i < 7: continue
        #if i > 15: break

        for iParagraph, element in enumerate(page_layout):
            if isinstance(element, LTTextContainer):

                #print("[%s]" % element.get_text())

                lines = list(element)

                # Before going into detection, fix new page problem:
                # On every new page there is a page number. Sometimes there is a empty line between
                # So do: Remove empty lines if first "text" line does NOT contains italic
                #iLine[0]
                if iParagraph == 0:

                    # Remove first line (Page Number)
                    lines.pop(0)

                    # Find first "text" line and remove others above
                    index = -1
                    for iLine, l in enumerate(lines):
                        if len(l.get_text().strip()) > 0:
                            # Is Text Line!
                            index = iLine
                            break

                    # If index >= 0, pop until text line
                    if index >= 0:
                        for j in range(0, index):
                            lines.pop(0)

                # Now go line by line and find kings and queens.
                for iLine, text_line in enumerate(lines):

                    text = text_line.get_text()
                    line_above = lines[iLine - 1]

                    first_char = list(text_line)[0]
                    
                    # Search for Italic. Then it could be a queen
                    if isinstance(first_char, LTChar) and ('italic' in first_char.fontname.lower()):
                        
                        # Stop, if still reading. EXCEPT: Line above was also italic (multilines)
                        if (
                            (reading and iLine == 0)
                            or (iLine > 0 and not('italic' in list(lines[iLine - 1])[0].fontname.lower())) #Multiline check
                        ):
                            # Maybe the italic is inline text? Check if Line above was empty line or not existing
                            if (
                                (iLine == 0)
                                or (line_above.get_text().strip() == '') # Inline italic Check
                            ):
                                paragraphs.append(textbuf)
                                reading = False
                                textbuf = ""
                        
                        # "Fake" check
                        is_fake = False
                        for fake in fakes:
                            if fake in text:
                                is_fake = True
                                break
                        reading = not is_fake
                    
                    # Stop Reading if Bold detected
                    if isinstance(first_char, LTChar) and ('bold' in first_char.fontname.lower()):
                        paragraphs.append(textbuf)
                        reading = False
                        textbuf = ""

                    # Fill Buffer if we are reading
                    if reading:
                        textbuf += text

    # If still reading, add to dict
    if reading:
        paragraphs.append(textbuf)

    return paragraphs

#p = read_pdf('../export/Files/2015-11-23-a7f10bd866ae4378bd85ca7a5f668eb3-332.pdf')

In [140]:
# Get all files
df = pd.read_csv(Path('../export/dokumente.csv'))

# Only "Protokoll"
df = df[df.dokument_kategorie == 'Protokoll']

# Loop file
for i, row in df.iterrows():

    # Create Name
    fname = Path('../export/extracts/%s' % row['_filename']).with_suffix('.json')

    if not fname.is_file():
        # Parse PDF
        paragraphs = read_pdf(Path('../export/Files/') / row['_filename'])

        # Save
        with open(fname, 'w', encoding='UTF-8') as f:
            f.write(json.dumps(paragraphs, ensure_ascii=False))

print("finito")

finito


In [134]:
#for x in p:
#    print("----")
#    print(x)

# Weg über Leerzeilen suchen - DEPRICATED

In [104]:
def read_pdf(f):
    i = 0

    textbuf = ""
    linebefore = ""
    reading = False

    paragraphs = []

    fakes = ['Detailberatung', 'Titel und Ingress', 'I. und II.', 'III.', 'Abstimmung', 'IV.',
    'Schlussabstimmung']

    pages = extract_pages(f)
    i = 0
    for page_layout in pages:
        print("-------")
        i += 1
        if i < 7: continue
        if i > 9: break

        for iParagraph, element in enumerate(page_layout):
            if isinstance(element, LTTextContainer):

                #print("[%s]" % element.get_text())

                lines = list(element)

                # If new Paragraph, its also a new speaker. Except, its a page break. 🤯
                if iParagraph > 0:
                    paragraphs.append(textbuf)
                    reading = False
                    textbuf = ""

                # Before going into detection, fix new page problem:
                # On every new page there is a page number. Sometimes there is a empty line between
                # So do: Rempve empty lines if first "text" line does NOT contains italic
                #iLine[0]
                if iParagraph == 0:

                    # Remove first line (Page Number)
                    lines.pop(0)

                    # If new first line is empty, check if next line contains italic. If not, pop.
                    #if (len(lines) > 0):
                    #    print("FIRST: %s" % lines[0].get_text().strip())

                    # Find first "text" line
                    index = -1
                    for iLine, l in enumerate(lines):
                        if len(l.get_text().strip()) > 0:
                            # Is Text Line!
                            first_char = list(l)[0]
                            if isinstance(first_char, LTChar) and not ('italic' in first_char.fontname.lower()):
                                index = iLine
                            break

                    # If index >= 0, pop until text line
                    if index >= 0:
                        print(index)
                        for j in range(0, index):
                            lines.pop(0)

                    #if (len(lines) > 0) and len(lines[0].get_text().strip()) == 0:
                    #    first_char = list(lines[1])[0]
                    #    if isinstance(first_char, LTChar) and not ('italic' in first_char.fontname.lower()):
                            # Pop first line
                    #        lines.pop(0)

                # Strip in last paragraph last (empty) lines

                for iLine, text_line in enumerate(lines):

                    text = text_line.get_text()
                    print("[%s]" % text)
                    #print(text)

                    # if first, remove page number and - if not italic - linebreaks
                    #if reading and (iParagraph == 0) and (iLine == 0):

                        # Replace Number
                        #text = re.sub('^\d*\s', '', text)


                    if reading:
                        if len(text.strip()) == 0:
                            # Stop reading
                            paragraphs.append(textbuf)
                            reading = False
                            textbuf = ""
                            print("!!stop")
                        else:
                            textbuf += text
                            
                    else:
                        # not Reading. Check first Character
                        for character in text_line:
                            if isinstance(character, LTChar):

                                if 'italic' in character.fontname.lower():
                                    # Is Italic

                                    # Check, if not fake italic
                                    is_fake = False
                                    for fake in fakes:
                                        if fake in text_line.get_text():
                                            is_fake = True
                                            break

                                    #if (not is_fake) and (len(linebefore) == 0):
                                    if not is_fake:
                                        # Start reading
                                        textbuf = text_line.get_text()
                                        reading = True
                            break

                    linebefore = text_line.get_text().strip().replace('\n', '')

    # If still reading, add to dict
    if reading:
        paragraphs.append(textbuf)

    return paragraphs

p = read_pdf('../export/Files/2015-11-23-a7f10bd866ae4378bd85ca7a5f668eb3-332.pdf')


-------
-------
-------
-------
-------
-------
-------
[1499 
]
[Schlussabstimmung 
]
[Der  Kantonsrat  beschliesst  mit  160  :  0  Stimmen  (bei  1  Enthal-
tung), die Fristerstreckung um ein Jahr für die Berichterstattung 
und Antragstellung zur Motion 217/2012 zu genehmigen.  
]
[Das Geschäft ist erledigt. 
]
[5. Bauprogramm der Staatsstrassen für die Jahre 2016–2018
Antrag des Regierungsrates vom 23. September 2015 und gleichlau-
tender Antrag der Kommission für Planung und Bau vom 27. Okto-
ber 2015  
Vorlage 5231 
]
[Ratspräsidentin  Theresia  Weber:  Ich  begrüsse  unsere  Volkswirt-
schaftsdirektorin, Regierungsrätin Carmen Walker Späh. 
]
[Erich  Bollinger  (SVP,  Rafz),  Präsident  der  Kommission  für  Planung 
und Bau (KPB):  Auch dieses Jahr wird der Kantonsrat noch vor den 
Budgetberatungen  das  Bauprogramm  der  Staatsstrassen  zur  Kenntnis 
nehmen. Für diejenigen, denen dieser Ablauf noch nicht geläufig ist, 
zitiere  ich  den  entsprechenden  Passus  aus  dem  Stra

In [105]:
def read_pdf(f):
    i = 0

    textbuf = ""
    linebefore = ""
    reading = False

    paragraphs = []

    fakes = ['Detailberatung', 'Titel und Ingress', 'I. und II.', 'III.', 'Abstimmung', 'IV.',
    'Schlussabstimmung']

    pages = extract_pages(f)
    i = 0
    for page_layout in pages:
        print("-------")
        i += 1

        for iParagraph, element in enumerate(page_layout):
            if isinstance(element, LTTextContainer):

                print("[%s]" % element.get_text())



p = read_pdf('../export/Files/2015-11-23-a7f10bd866ae4378bd85ca7a5f668eb3-332.pdf')


-------
[Protokoll des Zürcher Kantonsrates 
]
[1493 
]
[24. Sitzung, Montag, 23. November 2015, 8.15 Uhr 024
Vorsitz: Theresia Weber (SVP, Uetikon a. S.) 
]
[Verhandlungsgegenstände 
]
[1. Mitteilungen
]
[– Antworten auf Anfragen ......................................... Seite  1495
– Ratsprotokoll zur Einsichtnahme ............................ Seite  1495
]
[2. Wahl eines Mitglieds der Kommission für 
]
[Wirtschaft und Abgaben
für die aus der Kommission ausgetretene Regine
Sauter, Zürich
Antrag der Interfraktionellen Konferenz
KR-Nr. 290/2015 ........................................................ Seite  1496 
]
[3. Wahl eines Ersatzmitglieds des 
]
[Baurekursgerichts
für den zum Mitglied gewählten Jürg Trachsel,
Richterswil
Antrag der Interfraktionellen Konferenz
KR-Nr. 291/2015 ........................................................ Seite  1496 
]
[4. Fristerstreckung für die Berichterstattung und 
Antragstellung zur Motion KR-Nr. 217/2012 
betreffend Sozialversicherungsbeiträge 